In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split

# Veriyi okuyun
X = pd.read_csv('data/train.csv', index_col='Id') 
X_test = pd.read_csv('data/test.csv', index_col='Id')

# Hedef değişkeni eksik olan satırları çıkarın ve hedefi değişkeni ayırın
X.dropna(axis=0, subset=['SalePrice'], inplace=True)
y = X.SalePrice
X.drop(['SalePrice'], axis=1, inplace=True)

# Eksik değeri olan sütunları çıkarın
cols_with_missing = [col for col in X.columns if X[col].isnull().any()] 
X.drop(cols_with_missing, axis=1, inplace=True)
X_test.drop(cols_with_missing, axis=1, inplace=True)

# Eğitim ve doğrulama veri setlerini ayırın
X_train, X_valid, y_train, y_valid = train_test_split(X, y,train_size=0.8, test_size=0.2,random_state=0)

Verilerin ilk beş satırını yazdırmak için bir sonraki kod hücresini kullanın.

In [2]:
X_train.head()

,MSSubClass,MSZoning,LotArea,Street,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,...,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold,SaleType,SaleCondition
Id,,,,,,,,,,,,,,,,,,,,,
619,20,RL,11694,Pave,Reg,Lvl,AllPub,Inside,Gtl,NridgHt,...,108,0,0,260,0,0,7,2007,New,Partial
871,20,RL,6600,Pave,Reg,Lvl,AllPub,Inside,Gtl,NAmes,...,0,0,0,0,0,0,8,2009,WD,Normal
93,30,RL,13360,Pave,IR1,HLS,AllPub,Inside,Gtl,Crawfor,...,0,44,0,0,0,0,8,2009,WD,Normal
818,20,RL,13265,Pave,IR1,Lvl,AllPub,CulDSac,Gtl,Mitchel,...,59,0,0,0,0,0,7,2008,WD,Normal
303,20,RL,13704,Pave,IR1,Lvl,AllPub,Corner,Gtl,CollgCr,...,81,0,0,0,0,0,1,2006,WD,Normal


Veri setinin hem sayısal hem de kategorik değişkenler içerdiğine dikkat edin. Bir modeli eğitmeden önce kategorik verileri kodlamanız gerekecektir.

Farklı modelleri karşılaştırmak için, eğitimdeki aynı `score_dataset()` işlevini kullanacaksınız. Bu işlev, rastgele bir orman modelinden [ortalama mutlak hatayı](https://en.wikipedia.org/wiki/Mean_absolute_error) (MAE) bildirir.

In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error

# Farklı yaklaşımları karşılaştırmak için fonksiyon 
def score_dataset(X_train, X_valid, y_train, y_valid):
    # Random Forest modelini oluşturun ve eğitin
    model = RandomForestRegressor(n_estimators=100, random_state=0)
    model.fit(X_train, y_train)
    
    # Doğrulama seti üzerinde tahmin yapın
    preds = model.predict(X_valid)
    
    # Ortalama mutlak hata (MAE) hesaplayın ve döndürün
    return mean_absolute_error(y_valid, preds)

# Soru1 : Kategorik veriler içeren sütunları kaldırın

En basit yaklaşımla başlayacaksınız. Kategorik veriler içeren sütunları kaldırmak için `X_train` ve `X_valid` içindeki verileri önceden işlemek için aşağıdaki kod hücresini kullanın. Önceden işlenmiş DataFrames'i sırasıyla `drop_X_train` ve `drop_X_valid` olarak ayarlayın.

In [4]:
# Kategorik veri tipine sahip sütunları çıkar: eğitim ve doğrulama verisi için
# Eğitim verisinden 'object' tipindeki sütunları çıkar
drop_X_train = X_train.select_dtypes(exclude=["object"]) 
# Doğrulama verisinden 'object' tipindeki sütunları çıkar
drop_X_valid = X_valid.select_dtypes(exclude=["object"])  

Bu yaklaşım için MAE'yi almak üzere bir sonraki kod hücresini çalıştırın.

In [5]:
print("MAE from Approach 1 (Drop categorical variables):")
print(score_dataset(drop_X_train, drop_X_valid, y_train, y_valid))

MAE from Approach 1 (Drop categorical variables):
17837.82570776256


Sıralı kodlamaya geçmeden önce, veri kümesini inceleyeceğiz. Özellikle `'Condition2'` sütununa bakacağız. Aşağıdaki kod hücresi, hem eğitim hem de doğrulama kümelerindeki benzersiz girdileri yazdırır.

In [6]:
print("Unique values in 'Condition2' column in training data:", X_train['Condition2'].unique())
print("\nUnique values in 'Condition2' column in validation data:", X_valid['Condition2'].unique())

Unique values in 'Condition2' column in training data: ['Norm' 'PosA' 'Feedr' 'PosN' 'Artery' 'RRAe']

Unique values in 'Condition2' column in validation data: ['Norm' 'RRAn' 'RRNn' 'Artery' 'Feedr' 'PosN']


# Soru2: Sıralı kodlama

### Bölüm A

Şimdi şu kodu yazarsanız:
- eğitim verilerine ordinal bir kodlayıcı uydurun ve sonra
- hem eğitim hem de doğrulama verilerini dönüştürmek için kullanın,

bir hata alırsınız. Bunun neden böyle olduğunu görebiliyor musunuz? (_Bu soruyu cevaplamak için yukarıdaki çıktıyı kullanmanız gerekir._)

Bu, gerçek dünya verileriyle karşılaşacağınız yaygın bir sorundur ve bu sorunu çözmek için birçok yaklaşım vardır. Örneğin, yeni kategorilerle başa çıkmak için özel bir sıra kodlayıcı yazabilirsiniz. Ancak en basit yaklaşım, sorunlu kategorik sütunları bırakmaktır.

Sorunlu sütunları bir Python listesi `bad_label_cols`'a kaydetmek için aşağıdaki kod hücresini çalıştırın. Benzer şekilde, güvenli bir şekilde sıra kodlanabilen sütunlar `good_label_cols`'da saklanır.

In [8]:
# Eğitim verisindeki kategorik sütunlar
object_cols = [col for col in X_train.columns if X_train[col].dtype == "object"]

# Sıralı kodlama için uygun sütunlar
good_label_cols = [col for col in object_cols if 
                   set(X_valid[col]).issubset(set(X_train[col]))]
        
# Sıralı kodlama için uygun olmayan, veri setinden çıkarılacak sütunlar
bad_label_cols = list(set(object_cols) - set(good_label_cols))
        
print('Sıralı kodlanacak kategorik sütunlar:', good_label_cols)
print('\nVeri setinden çıkarılacak kategorik sütunlar:', bad_label_cols)

'''
good_label_cols:
Doğrulama setindeki değerler, eğitim setindekilerin bir alt kümesi olan sütunları bulur.

bad_label_cols:
Doğrulama setinde eğitim setinde olmayan değerler içeren sütunları bulur ve bunları çıkarır.
'''

Sıralı kodlanacak kategorik sütunlar: ['MSZoning', 'Street', 'LotShape', 'LandContour', 'Utilities', 'LotConfig', 'LandSlope', 'Neighborhood', 'Condition1', 'BldgType', 'HouseStyle', 'RoofStyle', 'Exterior1st', 'Exterior2nd', 'ExterQual', 'ExterCond', 'Foundation', 'Heating', 'HeatingQC', 'CentralAir', 'KitchenQual', 'PavedDrive', 'SaleType', 'SaleCondition']

Veri setinden çıkarılacak kategorik sütunlar: ['Functional', 'Condition2', 'RoofMatl']


'\ngood_label_cols:\nDoğrulama setindeki değerler, eğitim setindekilerin bir alt kümesi olan sütunları bulur.\n\nbad_label_cols:\nDoğrulama setinde eğitim setinde olmayan değerler içeren sütunları bulur ve bunları çıkarır.\n'

### Bölüm B

`X_train` ve `X_valid` içindeki verileri sıralı olarak kodlamak için bir sonraki kod hücresini kullanın. Önceden işlenmiş Veri Çerçevelerini sırasıyla `label_X_train` ve `label_X_valid` olarak ayarlayın.
- `bad_label_cols` içindeki kategorik sütunları veri kümesinden çıkarmak için aşağıda kod sağladık.
- `good_label_cols` içindeki kategorik sütunları sıralı olarak kodlamalısınız.

In [9]:
from sklearn.preprocessing import OrdinalEncoder

# Kodlanmayacak kategorik sütunları çıkar
label_X_train = X_train.drop(bad_label_cols, axis=1)
label_X_valid = X_valid.drop(bad_label_cols, axis=1)

# Sıralı kodlayıcıyı uygula
ordinal_encoder = OrdinalEncoder()

# Eğitim ve doğrulama verilerindeki uygun sütunları sıralı kodlama yöntemiyle dönüştür
label_X_train[good_label_cols] = ordinal_encoder.fit_transform(X_train[good_label_cols])
label_X_valid[good_label_cols] = ordinal_encoder.transform(X_valid[good_label_cols])

Bu yaklaşım için MAE'yi almak üzere bir sonraki kod hücresini çalıştırın.

In [10]:
print("MAE from Approach 2 (Ordinal Encoding):") 
print(score_dataset(label_X_train, label_X_valid, y_train, y_valid))

MAE from Approach 2 (Ordinal Encoding):
17098.01649543379


Şimdiye kadar kategorik değişkenlerle başa çıkmak için iki farklı yaklaşım denediniz. Ve kategorik verileri kodlamanın, veri kümesinden sütunları kaldırmaktan daha iyi sonuçlar verdiğini gördünüz.

Yakında, one hot encoding kodlamayı deneyeceksiniz. Ondan önce, ele almamız gereken ek bir konu var. Bir sonraki kod hücresini değişiklik yapmadan çalıştırarak başlayın.

In [11]:
# Kategorik verilere sahip her sütundaki benzersiz girişlerin sayısını alın
object_nunique = list(map(lambda col: X_train[col].nunique(), object_cols))

# Sütun adları ve benzersiz giriş sayıları ile bir sözlük oluşturun
d = dict(zip(object_cols, object_nunique))

# Benzersiz giriş sayısına göre sütunları artan sırayla yazdırın
sorted(d.items(), key=lambda x: x[1])

[('Street', 2),
 ('Utilities', 2),
 ('CentralAir', 2),
 ('LandSlope', 3),
 ('PavedDrive', 3),
 ('LotShape', 4),
 ('LandContour', 4),
 ('ExterQual', 4),
 ('KitchenQual', 4),
 ('MSZoning', 5),
 ('LotConfig', 5),
 ('BldgType', 5),
 ('ExterCond', 5),
 ('HeatingQC', 5),
 ('Condition2', 6),
 ('RoofStyle', 6),
 ('Foundation', 6),
 ('Heating', 6),
 ('Functional', 6),
 ('SaleCondition', 6),
 ('RoofMatl', 7),
 ('HouseStyle', 8),
 ('Condition1', 9),
 ('SaleType', 9),
 ('Exterior1st', 15),
 ('Exterior2nd', 16),
 ('Neighborhood', 25)]

# Soru 3: Kardinaliteyi araştırma

### Bölüm A

Yukarıdaki çıktı, kategorik verilerin bulunduğu her sütun için sütundaki benzersiz değerlerin sayısını gösterir. Örneğin, eğitim verilerindeki `'Sokak'' sütununun iki benzersiz değeri vardır: `'Grvl'' ve `'Pave'', sırasıyla çakıl yol ve asfalt yola karşılık gelir.

Bir kategorik değişkenin benzersiz giriş sayısına, o kategorik değişkenin **kardinalitesi** olarak atıfta bulunuruz. Örneğin, `'Sokak'' değişkeninin kardinalitesi 2'dir.

Aşağıdaki soruları yanıtlamak için yukarıdaki çıktıyı kullanın.

In [12]:
# Eğitim verisinde kardinalitesi 10'dan büyük olan kaç tane kategorik değişken var?
high_cardinality_numcols = 3  # (Kardinalitesi 10'dan büyük sütun sayısı)

# Eğitim verisindeki 'Neighborhood' değişkenini one-hot kodlamak için kaç sütun gerekli?
num_cols_neighborhood = 25  # ('Neighborhood' değişkeni için gereken one-hot sütun sayısı)

### Bölüm B

Çok sayıda satıra sahip büyük veri kümeleri için, tek sıcak kodlama veri kümesinin boyutunu büyük ölçüde genişletebilir. Bu nedenle, genellikle yalnızca nispeten düşük kardinaliteye sahip sütunları tek sıcak kodlayacağız. Daha sonra, yüksek kardinaliteye sahip sütunlar veri kümesinden çıkarılabilir veya sıralı kodlama kullanabiliriz.

Örnek olarak, 10.000 satıra sahip ve 100 benzersiz girişi olan bir kategorik sütun içeren bir veri kümesini ele alalım.

- Bu sütun karşılık gelen tek sıcak kodlamayla değiştirilirse, veri kümesine kaç giriş eklenir?

- Bunun yerine sütunu sıralı kodlamayla değiştirirsek, kaç giriş eklenir?

Cevaplarınızı aşağıdaki satırları doldurmak için kullanın.

In [13]:
# Sütunu one-hot kodlama ile değiştirdiğimizde veri setine kaç giriş eklenir?
OH_entries_added = 1e4 * 100 - 1e4  # (Yeni sütun sayısı * satır sayısı - orijinal sütun sayısı)

# Sütunu sıralı (ordinal) kodlama ile değiştirdiğimizde veri setine kaç giriş eklenir?
label_entries_added = 0  # (Sıralı kodlama ek sütun eklemez, sadece mevcut sütunu dönüştürür)

Sonra, tone-hot-wncoding kodlamayı deneyeceksiniz. Ancak, veri kümesindeki tüm kategorik değişkenleri kodlamak yerine, yalnızca kardinalitesi 10'dan küçük olan sütunlar için tek-sıcak kodlama oluşturacaksınız.

Aşağıdaki kod hücresini, `low_cardinality_cols`'u tek-sıcak kodlanacak sütunları içeren bir Python listesine ayarlamak için değişiklik yapmadan çalıştırın. Benzer şekilde, `high_cardinality_cols` veri kümesinden kaldırılacak kategorik sütunların bir listesini içerir.;

In [14]:
# One-hot kodlanacak sütunlar: Kardinalitesi 10'dan az olan kategorik sütunlar
low_cardinality_cols = [col for col in object_cols if X_train[col].nunique() < 10]

# Veri setinden çıkarılacak sütunlar: Kardinalitesi yüksek olan kategorik sütunlar
high_cardinality_cols = list(set(object_cols) - set(low_cardinality_cols))

# One-hot kodlanacak kategorik sütunları yazdır
print('One-hot kodlanacak kategorik sütunlar:', low_cardinality_cols)
# Veri setinden çıkarılacak kategorik sütunları yazdır
print('\nVeri setinden çıkarı')

One-hot kodlanacak kategorik sütunlar: ['MSZoning', 'Street', 'LotShape', 'LandContour', 'Utilities', 'LotConfig', 'LandSlope', 'Condition1', 'Condition2', 'BldgType', 'HouseStyle', 'RoofStyle', 'RoofMatl', 'ExterQual', 'ExterCond', 'Foundation', 'Heating', 'HeatingQC', 'CentralAir', 'KitchenQual', 'Functional', 'PavedDrive', 'SaleType', 'SaleCondition']

Veri setinden çıkarı


# Soru 4: One-Hot kodlama

`X_train` ve `X_valid` içindeki verileri tek sıcak kodlamak için bir sonraki kod hücresini kullanın. Önceden işlenmiş DataFrames'i sırasıyla `OH_X_train` ve `OH_X_valid` olarak ayarlayın.
- Veri setindeki kategorik sütunların tam listesi Python listesi `object_cols` içinde bulunabilir.
- `low_cardinalite_cols` içindeki kategorik sütunları yalnızca tek sıcak kodlamanız gerekir. Diğer tüm kategorik sütunlar veri setinden çıkarılmalıdır.

In [19]:
from sklearn.preprocessing import OneHotEncoder

# One-hot kodlayıcıyı her kategorik veri sütununa uygulayın
OH_encoder = OneHotEncoder(handle_unknown='ignore', sparse_output=False)
OH_cols_train = pd.DataFrame(OH_encoder.fit_transform(X_train[low_cardinality_cols]))
OH_cols_valid = pd.DataFrame(OH_encoder.transform(X_valid[low_cardinality_cols]))

# One-hot kodlama sırasında kaybolan indeksleri geri yükleyin
OH_cols_train.index = X_train.index
OH_cols_valid.index = X_valid.index

# Kategorik sütunları çıkarın (bunlar one-hot kodlama ile değiştirilecek)
num_X_train = X_train.drop(object_cols, axis=1)
num_X_valid = X_valid.drop(object_cols, axis=1)

# One-hot kodlanmış sütunları sayısal özelliklerle birleştirin
OH_X_train = pd.concat([num_X_train, OH_cols_train], axis=1)
OH_X_valid = pd.concat([num_X_valid, OH_cols_valid], axis=1)

OH_X_train.columns = OH_X_train.columns.astype(str)
OH_X_valid.columns = OH_X_valid.columns.astype(str)


Bu yaklaşım için MAE'yi almak üzere bir sonraki kod hücresini çalıştırın.

In [20]:
print("MAE from Approach 3 (One-Hot Encoding):") 
print(score_dataset(OH_X_train, OH_X_valid, y_train, y_valid))

MAE from Approach 3 (One-Hot Encoding):
17525.345719178084


In [ ]:
# (Optional) Your code here